In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch
import pandas as pd
from sklearn.metrics import classification_report
import gc


In [ ]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
# Free up GPU memory before loading
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Define model parameters
fine_tuned_model_name = "KaroKruse/lora_model_tuned_1"  # Your fine-tuned model on Hugging Face
max_seq_length = 2048  # Adjust based on your needs
dtype = None  # Choose dtype based on your GPU (e.g., torch.float16 for certain GPUs)
load_in_4bit = True  # Load with 4-bit quantization to reduce memory usage


In [ ]:
# Load the fine-tuned model and tokenizer using FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=fine_tuned_model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

meta-llama/Llama-3.1-8B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# Enable inference mode for Unsloth models
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
# Suppress warning by setting pad_token_id to eos_token_id
tokenizer.pad_token_id = tokenizer.eos_token_id

# Define the classification prompt template
classification_prompt = """
Du er en klassifikationsmodel, der skal afgøre, om følgende tekst indeholder identitetspolitik eller ej.
Tekst: {text}
Svar:
"""

classification_prompt1 = """
Du er en klassifikationsmodel, der skal afgøre, om følgende tekst indeholder identitetspolitik eller ej.

Klassifikation:
- Klassificér teksten som identitetspolitik, hvis den relaterer til rettigheder, repræsentation eller ligestilling af historisk marginaliserede grupper (f.eks., kvinder, LGBTQ+, kønsminoriteter, etniske og religiøse minoriteter).
- Klassificér teksten som ikke identitetspolitik, hvis den ikke indeholder sådanne elementer.

Læs teksten grundigt og afgør klassifikationen.

Tekst: {text}
Svar:
""" ## 16 forkerte forudsigelser alle samme identitetspolitik. 78/511.

classification_prompt3 = """
Du er en klassifikationsmodel, der skal afgøre, om følgende tekst indeholder identitetspolitisk indhold.

Klassifikation:
- Klassificér teksten som identitetspolitik, hvis den nævner, antyder, eller relaterer til rettigheder, repræsentation, eller ligestilling af historisk marginaliserede grupper (f.eks., kvinder, LGBTQ+, kønsminoriteter, etniske og religiøse minoriteter).
- Klassificér teksten som ikke identitetspolitik, hvis der ingen referencer til sådanne grupper eller emner findes i teksten.

Vigtigt: Selv en indirekte henvisning til identitetspolitik bør klassificeres som identitetspolitik.

Tekst: {text}
Svar:
""" # Samme som ovenstående


In [ ]:
# Function to classify a single text
def classify_text(text, max_length=1000):
    truncated_text = text[:max_length]
    prompt = classification_prompt1.format(text=truncated_text)

    # Tokenize the input prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the output
    output = model.generate(
        **inputs,
        max_new_tokens=6,
        temperature=0.1,
        top_k=1,
        top_p=0.1
    )

    # Decode and classify the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip().lower()
    print(f"Generated text: {generated_text}")

    svar_index = generated_text.find("svar:")

    if svar_index != -1:
        answer = generated_text[svar_index + len("svar:"):].strip()
        if answer in {"1", "1.0", "1. 0"}:
            return 1  # "identitetspolitik"
        elif answer in {"0", "0.0", "0. 0"}:
            return 0  # "ikke identitetspolitik"
        else:
            return 0

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load test dataset
file_path = '/content/drive/MyDrive/Speciale/Data_Pre_Model/2009.xlsx'
df = pd.read_excel(file_path, header=0)
df.rename(columns={"text": "Text"}, inplace=True)
df = df.fillna('')
print(df.head())

Mounted at /content/drive
            parid                                               Text  \
0  2009-01-13-3-1  Efter Det Radikale Venstres opfattelse er rege...   
1  2009-01-13-3-2  Og det er sådan set begrundelsen for forespørg...   
2  2009-01-13-5-1  Jeg takker for ordet. Jeg vil også gerne start...   
3  2009-01-13-5-2  Det er bl.a. derfor, vi i disse dage bruger en...   
4  2009-01-13-5-3  Ideen med at have en 2015-plan og tidligere en...   

                  speaker group  chair  parawordcount         src  year  
0  Morten Helveg Petersen    RV  False             69  parlspeech  2009  
1  Morten Helveg Petersen    RV  False             39  parlspeech  2009  
2    Lars Løkke Rasmussen     V  False            135  parlspeech  2009  
3    Lars Løkke Rasmussen     V  False             93  parlspeech  2009  
4    Lars Løkke Rasmussen     V  False            106  parlspeech  2009  


In [ ]:
# Classify each text and add predictions to new column
df['Predicted_Label'] = df['Text'].apply(classify_text)

Streaming af output blev afkortet til de sidste 5000 linjer.

læs teksten grundigt og afgør klassifikationen.

tekst: så kan ordføreren ikke klart tilkendegive, at det ville være rigtigst i forhold til at skaffe arbejdspladser, hvis man brugte penge på offentlige investeringer nu i stedet for at give skattelettelser, som altså går til flyrejser og alt muligt andet?
svar:
0.0
Generated text: du er en klassifikationsmodel, der skal afgøre, om følgende tekst indeholder identitetspolitik eller ej.

klassifikation:
- klassificér teksten som identitetspolitik, hvis den relaterer til rettigheder, repræsentation eller ligestilling af historisk marginaliserede grupper (f.eks., kvinder, lgbtq+, kønsminoriteter, etniske og religiøse minoriteter).
- klassificér teksten som ikke identitetspolitik, hvis den ikke indeholder sådanne elementer.

læs teksten grundigt og afgør klassifikationen.

tekst: for det første vil jeg sige, at vi i det konservative folkeparti vil fortsætte med at medvirke til at s

In [ ]:
# Save the updated dataset with predictions
output_file_path = '/content/drive/MyDrive/Speciale/Data_Post_Model/2009_with_Predictions.xlsx'
df.to_excel(output_file_path, index=False)
print(f"File saved at {output_file_path}")

File saved at /content/drive/MyDrive/Speciale/Data_Post_Model/2009_with_Predictions.xlsx


In [ ]:
df['Predicted_Label']

,Predicted_Label
0,0
1,0
2,0
3,0
4,0
...,...
63509,0
63510,0
63511,0
63512,0


In [ ]:
import smtplib
from email.mime.text import MIMEText
from google.colab import runtime

# Indstil din e-mail information
sender_email = "topsej7@gmail.com"  # Din Gmail-adresse
receiver_email = "matilde4567@hotmail.com"  # Modtageradressen
app_password = "your_app_password"  # Din app-adgangskode fra Gmail

# Opsæt e-mailens indhold
subject = "Google Colab-kørsel færdig"
body = "Din Google Colab-kørsel er nu afsluttet."

msg = MIMEText(body)
msg["Subject"] = subject
msg["From"] = sender_email
msg["To"] = receiver_email

# Send e-mailen
try:
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender_email, app_password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
    print("E-mail sendt!")
except Exception as e:
    print("Fejl ved afsendelse af e-mail:", e)

# Stop runtime-sessionen
runtime.unassign()
